# Hidden shift algorithm


[Bravyi and Gosset](https://arxiv.org/abs/1601.07601v3) and [Bravyi *et. al*](https://arxiv.org/abs/1808.00128v2) state that the hidden shift problem is particularly suited to benchmark their quantum circuit simulator because:

1. the output is deterministic;
and
2. the $T$ count of the algorithm is easily controlled by choosing a suitable bent function.

Following the description presented in the Appendix F of [[Bravyi and Gosset]](https://arxiv.org/abs/1601.07601v3), I will implement the hidden shift algorithm here as well.

---

This notebook was used to generate the following samples of the hidden-shift algorithm with the goal of benchmarking our code:

* $t=14$:
    1. $n=10$;
    2. $n=14$;
    3. $n=18$;
    4. $n=22$;
    5. $n=28$;
    6. $n=32$;
    

* $t=28$:
    1. $n=14$;
    2. $n=18$;
    3. $n=22$;
    4. $n=28$;
    5. $n=32$;
    6. $n=36$.

In [1]:
import qiskit
import random

from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.tools.visualization import plot_histogram

In [2]:
def oracle_g(n, n_ccz, g):
    """Oragel O_g

    Args:
        n (int): Number of qubits in the quantum circuit
        n_ccz (int): Number of Toffoli (or CZZ) gates
        g (int): Number of Z and CZ gates in the {Z,CZ}-layers

    Returns:
        QuantumCircuit: The O_g oracle quantum circuit
    """
    qr_g = qiskit.QuantumRegister(int(n / 2))

    og = qiskit.QuantumCircuit(qr_g, name='oracle_g')

    gates = ['z', 'cz']
    qubits = [i for i in range(int(n / 2))]
    for i in range(g):
        gate = random.sample(gates, 1)
        if gate[0] == 'z':
            target = random.sample(qubits, 1)
            og.s(qr_g[target])
            og.s(qr_g[target])
        elif gate[0] == 'cz':
            control, target = random.sample(qubits, 2)
            og.h(qr_g[target])
            og.cx(qr_g[control], qr_g[target])
            og.h(qr_g[target])

    for i in range(n_ccz):
        c1, c2, t = random.sample(qubits, 3)
        og.h(qr_g[t])
        og.ccx(qr_g[c1], qr_g[c2], qr_g[t])
        og.h(qr_g[t])
        for i in range(g):
            gate = random.sample(gates, 1)
            if gate[0] == 'z':
                target = random.sample(qubits, 1)
                og.s(qr_g[target])
                og.s(qr_g[target])
            elif gate[0] == 'cz':
                control, target = random.sample(qubits, 2)
                og.h(qr_g[target])
                og.cx(qr_g[control], qr_g[target])
                og.h(qr_g[target])

    return og


def oracle_f(n, og):
    """Oracle O_f

    Args:
        n (int): The number of qubits in the quantum circuit
        og (QuantumCircuit): The O_g oracle quantum circuit

    Returns:
        QuantumCircuit: The O_f oracle quantum circuit
    """
    qr_f = qiskit.QuantumRegister(n)

    of = qiskit.QuantumCircuit(qr_f, name='oracle_f')

    of = of.compose(og, qubits=qr_f[0:int(n / 2)])  # compose

    for i in range(int(n / 2)):
        of.h(qr_f[i + int(n / 2)])
        of.cx(qr_f[i], qr_f[i + int(n / 2)])
        of.h(qr_f[i + int(n / 2)])

    return of


def oracle_fp(n, og):
    """Oracle O_f^{\prime}

    Args:
        n (int): The number of qubits in the quantum circuit
        og (QuantumCircuit): The O_g oracle quantum circuit

    Returns:
        QuantumCircuit: The O_f^{\prime} oracle quantum circuit
    """
    qr_fp = qiskit.QuantumRegister(n)

    ofp = qiskit.QuantumCircuit(qr_fp, name='oracle_fp')

    for i in range(n):
        if s[i] == 1:
            ofp.s(qr_fp[i])
            ofp.s(qr_fp[i])

    ofp = ofp.compose(og, qubits=qr_fp[int(n / 2):n])  # compose

    for i in range(int(n / 2)):
        ofp.h(qr_fp[i + int(n / 2)])
        ofp.cx(qr_fp[i], qr_fp[i + int(n / 2)])
        ofp.h(qr_fp[i + int(n / 2)])

    return ofp


## $T$-count: $t=14$

### $n=10$


In [3]:
n = 10

In [4]:
# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

The "hidden" string is:  1001110010


In [5]:
n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

#### Remarks:

The generated quantum circuits will have the following properties:

* $n=10$ qubits,
* $800$ gates from the $\{ Z, \, CZ \}$ set (since $g=200$),
* $14$ $T$ gates,
and
* $w=10$ measurements.

The output of the computation is **deterministic** and corresponds to the string $s \in \mathbb{F}^{10}_2$ generated at random.

We want to export the randomly generated quantum circuits to a `.qasm` file and use them as input for our compilation code.

Given the way the compilation code is written, we will require a new python code which writes the circuit in terms of the allowed Clifford generators $H$, $S$ and $CX$, allowed to be input to the compilation.

In the form produced by the cells above, the circuits will not be valid inputs for the compilation code.
So we will need to do that pre-processing beforehand.

To simplify this task, we made sure that the oracles are immediately written in terms of the desired Clifford generators ($Z=S\cdot S$ and $CZ= (I\otimes H) CX_{12} (I \otimes H)$), such that we will **only need to worry with the decomposition of the Toffoli gate**. This makes the processing of the `.qasm` file simpler as we need only work through that decomposotion.

#### <span style=color:green>Sample generation!</span>
    

In [9]:
def sample_gen(nr_samples, n, n_ccz, s_string):
    nr_samples = 100
    
    for circuit in range(nr_samples):
        og = oracle_g(n, n_ccz, g)
        of = oracle_f(n, og)
        ofp = oracle_fp(n, og)
        
        qr = qiskit.QuantumRegister(n, 'qr')
        cr = qiskit.ClassicalRegister(n, 'cr')
        
        qc = qiskit.QuantumCircuit(qr, cr, name='U')
        
        for i in range(n):
            qc.h(i)
        
        qc = qc.compose(of, qr[0: n])
        
        for i in range(n):
            qc.h(i)
        
        qc = qc.compose(ofp, qr[0: n])
        
        for i in range(n):
            qc.h(i)
            qc.measure(i, n - 1 - i)
        
        qc.qasm(
            formatted=False,
            filename=
            f'tcount{n_ccz*14}--n{n}/s{s_string}/_samples/HSA{circuit}.qasm')

In [7]:
def Toff_decomp(nr_samples, n, n_ccz, s_string):
    for circuit in range(nr_samples):
        with open(
                f'tcount{n_ccz*14}--n{n}/s{s_string}/_samples/HSA{circuit}.qasm'
        ) as file_object:
            file_lines = file_object.readlines()

            clines = file_lines.copy()
            const = 0
            for line in clines:
                if line.startswith('ccx'):
                    index = clines.index(line) + const
                    a = line.partition('[')
                    b = a[2].partition(']')
                    ctrl_qubit1 = int(b[0])

                    c = b[2].partition('[')
                    d = c[2].partition(']')
                    ctrl_qubit2 = int(d[0])

                    e = d[2].partition('[')
                    f = e[2].partition(']')
                    target_qubit = int(f[0])

                    file_lines.remove(line)
                    # Toffoli decomposition:

                    # (1) controlled-not gate between qubits 1 and 2:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit1}],qr[{ctrl_qubit2}];\n')
                    # (2) T gate on qubit 1:
                    file_lines.insert(index, f't qr[{ctrl_qubit1}];\n')
                    # (3) T^dagger on qubit 2:
                    file_lines.insert(index, f's qr[{ctrl_qubit2}];\n')
                    file_lines.insert(index, f's qr[{ctrl_qubit2}];\n')
                    file_lines.insert(index, f's qr[{ctrl_qubit2}];\n')
                    file_lines.insert(index, f't qr[{ctrl_qubit2}];\n')
                    # (4) controlled-not gate between qubits 1 and 2:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit1}],qr[{ctrl_qubit2}];\n')
                    # (5) H gate on qubit 3:
                    file_lines.insert(index, f'h qr[{target_qubit}];\n')
                    # (6) T gates on qubits 2 and 3:
                    file_lines.insert(index, f't qr[{ctrl_qubit2}];\n')
                    file_lines.insert(index, f't qr[{target_qubit}];\n')
                    # (7) controlled-not gate between qubits 1 and 3:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit1}],qr[{target_qubit}];\n')
                    # (8) T^dagger on qubit 3:
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f't qr[{target_qubit}];\n')
                    # (9) controlled-not gate between qubits 2 and 3:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit2}],qr[{target_qubit}];\n')
                    # (10) T gate on qubit 3:
                    file_lines.insert(index, f't qr[{target_qubit}];\n')
                    # (11) controlled-not gate between qubits 1 and 3:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit1}],qr[{target_qubit}];\n')
                    # (12) T^dagger on qubit 3:
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f's qr[{target_qubit}];\n')
                    file_lines.insert(index, f't qr[{target_qubit}];\n')
                    # (13) controlled-not gate between qubits 2 and 3:
                    file_lines.insert(
                        index, f'cx qr[{ctrl_qubit2}],qr[{target_qubit}];\n')
                    # (14) H gate on qubit 3:
                    file_lines.insert(index, f'h qr[{target_qubit}];\n')
                    # This concludes the Toffoli decomposition
                    const += 23

        with open(
                f'tcount{n_ccz*14}--n{n}/s{s_string}/_samples/HSA{circuit}-input.qasm',
                'w') as file_object:
            for line in file_lines:
                file_object.write(line)

In [11]:
nr_samples = 100

sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=14$


In [12]:
n = 14

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  10110011100001


In [14]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=18$


In [15]:
n = 18

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  000101001000100010


In [17]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=22$


In [18]:
n = 22

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  1101111111111010011111


In [19]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=28$


In [20]:
n = 28

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  0000011010101101001001010100


In [22]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n=32$


In [23]:
n = 32

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 1 # implies t_count = 14
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  11101100001000010000011011111101


In [24]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

## $t=28$

### $n=14$

In [28]:
n = 14

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  00010110010011


In [30]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 18$

In [31]:
n = 18

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  110011111110110000


In [32]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 22$

In [33]:
n = 22

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  0011010010000110101000


In [34]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 28$

In [35]:
n = 28

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  1001011101011110011110101010


In [36]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 32$

In [37]:
n = 32

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  00000111001001011001110001001100


In [38]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

### $n= 36$

In [39]:
n = 36

# Generating a random hidden string

s = []
for i in range(n):
    j = random.randint(0, 1)
    s.append(j)

s_string = ''.join(str(j) for j in s)
print('The "hidden" string is: ', s_string)

n_ccz = 2 # implies t_count = 28
g = 200 # number of Z and CZ gates in the {Z,CZ}-layers in-between Toffoli gates 

nr_samples = 100

The "hidden" string is:  110010111011111100101101111101110111


In [41]:
sample_gen(nr_samples, n, n_ccz, s_string)
Toff_decomp(nr_samples, n, n_ccz, s_string)

---

## Importing some circuits to evaluate the depths:


In [17]:
samples = [f'tcount14--n32/s11101100001000010000011011111101/_samples/HSA{i}-input.qasm' for i in range(10)]

for i in range(10):
    imported_qc = qiskit.QuantumCircuit.from_qasm_file(samples[i])
    print(imported_qc.depth())

367
412
377
407
375
399
369
418
395
407


---